In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pickle
import pendulum
import requests
import json
import threading
from datetime import timedelta
import pandas as pd
from azure.cosmos import CosmosClient
import time
import numpy as np
import joblib

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

In [2]:
#model = joblib.load('model.pkl')
# model

### Import model from Blob Storage

In [3]:
connection_str = 'DefaultEndpointsProtocol=https;AccountName=joblove;AccountKey=6vRDCWJb8CfB1FAPvnUlubd4Kcxa75Mtut38aja0R2u293V/JucMzw72lR1aslcJwr2EYQWtlO4ybSAzd4K8fA==;EndpointSuffix=core.windows.net'
container_name = 'joblove'
azure_target_path = "model.pkl"

In [4]:
blob_client = BlobClient.from_connection_string(connection_str, container_name,azure_target_path)

In [5]:
x = blob_client.download_blob().readall()

In [6]:
model = pickle.loads(x)
model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('7',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('standardscalerwrapper',
                                                                                                                <azureml.automl.runtime.shared.mod...
                                                

In [7]:
# model.forecast()

### Query Power data

In [8]:
# CosmosDB
cosmos_url = "https://altocosmos.documents.azure.com:443/"
cosmos_key = "W6BRohNWpaaOFAyFMxW5NPGuHCDUHTNwQwXDVOyvMeDxLEtKBE8oF62GoWH2j4xUa3td5jIh6ljt8EVf5lmNdw=="
client = CosmosClient(cosmos_url, credential=cosmos_key)
database_name = 'altonucminteldb'
database = client.get_database_client(database_name)
container_name = 'iotcontainer'
container = database.get_container_client(container_name)
print(container)

def get_dataset_from_cosmosdb():
    # Get dataset from cosmosdb
    try:
        last_day_of_last_month_date = pendulum.now('Asia/Bangkok').subtract(months=1).end_of('month').to_date_string()
        first_day_of_next_month_date = pendulum.now('Asia/Bangkok').add(months=1).start_of('month').to_date_string()
        query_string = f"SELECT c.timestamp, c.device_id, c.subdevice_idx, c.location, c.subdevice_name, \
        c.power, c.gatewayid FROM c WHERE c.gatewayid='altonucmintelmonitor' AND c.location='main_energy/iot_devices' \
        AND c.subdevice_name='total' AND c.timestamp >= '2021-06-01' \
        AND c.timestamp < '{first_day_of_next_month_date}' \
        ORDER BY c.timestamp ASC"
        time1 = time.time()
        results = []

        try:
            for item in container.query_items(
                    query=query_string,
                    enable_cross_partition_query=True):
                #print(json.dumps(item, indent=True)) 
                results.append(item)
        except Exception as e:
            print("no data")
            print(e)

        print(f"Query time [second]: {time.time() - time1:.3f}")
        df = pd.DataFrame(results)
        print("finish get dataset from cosmosdb function")
        return df
    except Exception as e:
        print(f"[Error] get dataset from cosmosdb function: {e}")

dfe = get_dataset_from_cosmosdb()

<ContainerProxy [dbs/altonucminteldb/colls/iotcontainer]>
Query time [second]: 152.001
finish get dataset from cosmosdb function


In [9]:
def elec_dataprep(df):
    df['datetime'] = df['timestamp'].apply(lambda x: pendulum.parse(x).add(hours=7).to_datetime_string())
    df.loc[:, 'datetime'] = pd.to_datetime(df.loc[:, 'datetime'])
    df = df[['datetime', 'power']]
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    df = df.resample('30min').mean()
    return df

In [10]:
dfe2 = elec_dataprep(dfe)

In [11]:
dfe2.head(3)

,power
datetime,
2021-06-01 07:00:00,24.28
2021-06-01 07:30:00,31.04
2021-06-01 08:00:00,23.98


### Query Weather data

In [12]:
def get_outdoor_weather_data():
    dt = pendulum.now('Asia/Bangkok')
    _valid_time = pendulum.datetime(2021, 2, 1, tz='Asia/Bangkok')
    _end_time = pendulum.today('Asia/Bangkok')
    Data = {}

    def worker(dt):
        a = requests.get("https://api.weather.com/v1/location/VTBD:9:TH/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate={}".format(dt))
        b = json.loads(a.text)
        c = pd.DataFrame(b["observations"])
        Data[dt] = c

    num_th = 20
    while _end_time > _valid_time:

        threads = []        
        for i in range(num_th):            
            if _end_time <= _valid_time:
                break

            dt = _valid_time.strftime("%Y%m%d")
            t = threading.Thread(target=worker, args=(dt,))
            threads.append(t)       
            _valid_time = _valid_time.add(days=1)

        for t in threads:
            t.start()           
        for t in threads:
            t.join()

    df = pd.concat([Data[k] for k in Data], axis=0)

    # Remove columns with more than 90% data as null.
    cols_to_delete = df.columns[df.isnull().sum()/len(df) > .90]
    df.drop(cols_to_delete, axis = 1, inplace = True)

    df = df.sort_values(by="valid_time_gmt")

    df["datetime"] = pd.to_datetime(df["valid_time_gmt"].astype(int)*1e9) + timedelta(hours=7)
    df = df.reset_index(drop=True)
    df = df.set_index("datetime")

    # Drop irrelevant columns
    columns = ['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name']
    df.drop(columns=columns, inplace=True)

    return df

In [13]:
dfw = get_outdoor_weather_data()
dfw.head(3)

,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,wx_phrase,dewPt,heat_index,rh,pressure,vis,wc,wdir,wdir_cardinal,wspd,uv_desc,feels_like,uv_index,clds
datetime,,,,,,,,,,,,,,,,,,,
2021-02-01 00:00:00,1612112400,N,79,33,3300,Fair,68,81,69,29.93,3.00,79,nan,CALM,0,Low,81,0,FEW
2021-02-01 00:30:00,1612114200,N,79,33,3300,Fair,68,81,69,29.93,3.00,79,nan,CALM,0,Low,81,0,FEW
2021-02-01 01:00:00,1612116000,N,77,33,3300,Fair,68,79,74,29.93,3.00,77,nan,CALM,0,Low,79,0,CLR


### Merge both datasets

In [14]:
def merge(df1, df2):
    df = pd.merge(df1, df2, left_index=True, right_index=True)
    columns = ['power', 'temp', 'rh']
    
    return df[columns]

In [15]:
dff = merge(dfe2, dfw)
dff.head(3)

,power,temp,rh
datetime,,,
2021-06-01 07:00:00,24.28,84,79
2021-06-01 07:30:00,31.04,86,74
2021-06-01 08:00:00,23.98,88,70


### Prepare X_future.

In [16]:
def historical_api():

    dt = pendulum.now('Asia/Bangkok')
    # _valid_time = pendulum.today(tz='Asia/Bangkok')
    _valid_time = pendulum.today(tz='Asia/Bangkok')
    _end_time = pendulum.tomorrow(tz='Asia/Bangkok')
    Data = {}

    def worker(dt):
        a = requests.get("https://api.weather.com/v1/location/VTBD:9:TH/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate={}".format(dt))
        b = json.loads(a.text)
        c = pd.DataFrame(b["observations"])
        Data[dt] = c

    num_th = 20
    while _end_time > _valid_time:

        threads = []        
        for i in range(num_th):            
            if _end_time <= _valid_time:
                break

            dt = _valid_time.strftime("%Y%m%d")
            t = threading.Thread(target=worker, args=(dt,))
            threads.append(t)       
            _valid_time = _valid_time.add(days=1)

        for t in threads:
            t.start()           
        for t in threads:
            t.join()

    df = pd.DataFrame()
    df = pd.concat([Data[k] for k in Data], axis=0)

    # Remove columns with more than 90% data as null.
    cols_to_delete = df.columns[df.isnull().sum()/len(df) > .90]
    df.drop(cols_to_delete, axis = 1, inplace = True)

    df = df.sort_values(by="valid_time_gmt")

    df["datetime"] = pd.to_datetime(df["valid_time_gmt"].astype(int)*1e9) + timedelta(hours=7)
    df = df.reset_index(drop=True)
    df = df.set_index("datetime")

    df = df[['temp', 'rh']]
    df['temp'] = df['temp'].astype('float64')
    df['rh'] = df['rh'].astype('float64')
    return df

df_future1 = historical_api()

In [17]:
def forecast_api():
    link = 'https://api.weather.com/v1/location/VTBD:9:TH/forecast/hourly/24hour.json?units=e&language=en-US&apiKey=6532d6454b8aa370768e63d6ba5a832e'
    res = requests.get(link)
    print(json.loads(res.text)['metadata'])
    df = pd.DataFrame(json.loads(res.text)['forecasts'])
    df['datetime'] = pd.to_datetime(df['fcst_valid_local'].map(lambda x: pendulum.parse(x).to_datetime_string()))
    # print(df.columns)
    columns = ['datetime', 'temp', 'rh']
    df = df[columns]
    df = df.set_index('datetime')
    df = df.resample('30min').interpolate()
    return df

df_future2 = forecast_api()

{'language': 'en-US', 'transaction_id': '1625388373128:-1242219018', 'version': '1', 'location_id': 'VTBD:9:TH', 'units': 'e', 'expire_time_gmt': 1625388621, 'status_code': 200}


In [49]:
def concat_then_resample_df(df1, df2):

    df = pd.concat([df1,df2])
    df = df.resample('30min').interpolate()
    today = pendulum.today('Asia/Bangkok').to_date_string()
    df= df.loc[today]
    yesterday = pendulum.yesterday('Asia/Bangkok').to_date_string()
    #df = df.drop(columns = ['rh'], axis =1)
    return df.reset_index()

X_future = concat_then_resample_df(df_future1, df_future2)
X_future.head(3)

,datetime,temp,rh
0,2021-07-04 00:00:00,86.00,74.00
1,2021-07-04 00:30:00,86.00,74.00
2,2021-07-04 01:00:00,86.00,74.00


In [50]:
X_future.iloc[0]

datetime    2021-07-04 00:00:00
temp                      86.00
rh                        74.00
Name: 0, dtype: object

### Prepare y_future with np.NaN.

In [51]:
y_future = np.empty(len(X_future))
y_future[:] = np.NaN
y_future

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [52]:
# X_future.to_csv (r'x_f.csv', index = False, header=True)

In [38]:
model.target_column_name

'_automl_target_col'

In [61]:
# X_future['_automl_target_col'] = np.NaN
X_future.head(3)

,datetime,temp,rh
0,2021-07-04 00:00:00,86.00,74.00
1,2021-07-04 00:30:00,86.00,74.00
2,2021-07-04 01:00:00,86.00,74.00


In [62]:
input_sample = pd.DataFrame({"datetime": pd.Series(["2000-1-1"], dtype="datetime64[ns]"), "temp": pd.Series([0], dtype="int64"), "rh": pd.Series([0], dtype="int64")})
@input_schema('data', PandasParameterType(input_sample, enforce_shape=False))
def run(data):
    try:
        result = model.forecast(data, y_future)
    except Exception as e:
        result = str(e)
        print(json.dumps({"error": result}))

In [65]:
x = model.forecast(X_future, ignore_data_errors=True)

UntrainedModelException: UntrainedModelException:
	Message: UntrainedModelException: No target imputers were found in TimeSeriesTransformer.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "System",
        "message": "UntrainedModelException: No target imputers were found in TimeSeriesTransformer.",
        "target": "ForecastingPipelineWrapper",
        "reference_code": "ForecastingPipelineWrapper"
    }
}